In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [3]:
embed = hub.Module(module_url)

In [4]:
read = pd.read_csv('dataset/article_data_newyork', delimiter=',', header=None)
data_articles = read.iloc[:,:].values

In [5]:
read = pd.read_csv('dataset/table_data_newyork', delimiter=',', header=None)
data_tables = read.iloc[:,:].values

## creating the articles embedings

In [50]:
articles_title = []
articles_id = []

for article_url, article_page_title, article_main_passage, table_url in tqdm(data_articles):
    
    catch = article_page_title +" "+article_main_passage
    
    articles_id.append(table_url)
    
    articles_title.append(catch[:1000])

100%|██████████| 148/148 [00:00<00:00, 25957.89it/s]


In [51]:
with tf.Session() as sess:
    
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            
    embedding_articles = sess.run(embed(articles_title))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [52]:
article_dense_vector = []

for current_embedding in embedding_articles:
    
    article_dense_vector.append(current_embedding)

In [53]:
len(article_dense_vector)

148

## creating the tables embeddings

In [38]:
tables_title = []

for current_table in tqdm(data_tables):
    
    table_title = str(current_table[1])
    
    table_page_content = str(current_table[2])
    
    catch = str(current_table[1])+" "+str(current_table[2])
    
    tables_title.append(catch[:1000])

100%|██████████| 52426/52426 [00:00<00:00, 185797.49it/s]


In [39]:
with tf.Session() as sess:
    
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            
    embedding_tables = sess.run(embed(tables_title))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [40]:
tables_dense_vector = []

for current_embedding in embedding_tables:
    
    tables_dense_vector.append(current_embedding)

In [41]:
len(tables_dense_vector)

52426

## methods for getting the ranked tables and save the accuracy

In [16]:
def get_id_ranked_tables(top_k,distance_vector):

    id_ranked_tables = []

    for current_top_k in top_k:
        
        index = np.where(distance_vector == current_top_k)
         
        index_colummun = index[0][0]
        
        id_ranked_tables.append(data_tables[index_colummun][0])

    return id_ranked_tables

In [17]:
def get_accuracy(id_ranked_tables, id_query_goal):

    accuracy = 0

    for id_table in id_ranked_tables:
    
        if id_table == id_query_goal:
    
            accuracy = 1
            
            break;

    return accuracy

In [18]:
def save_accuracy(k,accuracy):
    
    if k == 1:
            
        average_top1.append(accuracy)
        
    if k == 5:
            
        average_top5.append(accuracy)
        
    if k == 10:
            
        average_top10.append(accuracy)
        
    if k == 100:
            
        average_top100.append(accuracy)
        
    if k == 1000:
            
        average_top1000.append(accuracy)

## computing the cosine similarity

In [54]:
average_top1 = []
average_top5 = []
average_top10 = []
average_top100 = []
average_top1000 = []

top_k = [5]

for i in tqdm(range(len(article_dense_vector))):
    
    distance_vector = pairwise_distances(article_dense_vector[i].reshape(1,512), tables_dense_vector, metric='cosine')
    
    id_query_goal = articles_id[i]
    
    for accuracy_k in top_k:
        
        count_top_tables = accuracy_k
        
        top_k_rank = nsmallest(count_top_tables, distance_vector[0])
    
        id_ranked_tables = get_id_ranked_tables(top_k_rank,distance_vector[0])
        
        accuracy_value = get_accuracy(id_ranked_tables,id_query_goal)
        
        #save the accuracy on the list
        save_accuracy(accuracy_k,accuracy_value)

100%|██████████| 148/148 [01:01<00:00,  2.42it/s]


In [55]:
print(str(round(np.mean(average_top1),4)))
print(str(round(np.mean(average_top5),4)))
print(str(round(np.mean(average_top10),4)))
print(str(round(np.mean(average_top100),4)))
print(str(round(np.mean(average_top1000),4)))

nan
0.4662
nan
nan
nan


/home/lss9/anaconda3/envs/virtual_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/lss9/anaconda3/envs/virtual_env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
